In [2]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import calendar
import math
import isodate

# Input CSV Here:

In [3]:
df = pd.read_csv('finalesterest.csv')
df.tail()

,Unnamed: 0.1,Unnamed: 0,id,channel_id,category_id,country,channel_viewCount,channel_subscriberCount,channel_videoCount,title,published,views,duration
21425,775,0,8Kn4yT4BdNg,UCHMrZNfXcc9VsOvINjd2E6g,10.0,NaN,770.0,3.0,36.0,Wildcoast Drive,2022-11-09T02:30:47Z,46,PT3M17S
21426,776,0,yiD9KHPqoGM,UC5cq3LFBx3QFpVhW7nwJLwA,NaN,NaN,NaN,NaN,NaN,BeamNG Drive Crashes #032 #shorts #shortsvideo...,2022-11-09T00:30:13Z,378,PT46S
21427,777,0,iSMzxxNZjDo,UCBKSNxU0iWKmMfp_TtJHBAA,NaN,NaN,NaN,NaN,NaN,"Drive News | Tesla Model 3, Model X | Փարթ 1",2022-11-08T17:00:08Z,19703,NaN
21428,778,0,NhcDrbPYwLo,UCB8wO7_QO9eZOV63oATZ35A,NaN,NaN,NaN,NaN,NaN,NYC Cross-Borough Drive | New York City 4K,2022-11-07T21:15:39Z,6787,NaN
21429,779,0,s3OjszLHL34,UCUU5GH6oM2qCBjgB3IyQfYQ,NaN,NaN,NaN,NaN,NaN,drive all night,2022-11-07T15:24:33Z,200,NaN


In [16]:
type(df.duration[21429])

float

In [17]:
def duration_to_seconds(x):
    if x == 'P0D':
        return None
    elif type(x) == float:
        return None
    else:
        try:
            return isodate.parse_duration(x).total_seconds()
        except:
            print(x)

In [19]:
df['duration'] = df['duration'].apply(duration_to_seconds)

In [23]:
df.published

0        2022-11-11T13:00:09Z
1        2022-11-11T12:00:16Z
2        2022-11-11T12:00:22Z
3        2022-11-11T10:12:10Z
4        2022-11-11T09:00:09Z
                 ...         
21425    2022-11-09T02:30:47Z
21426    2022-11-09T00:30:13Z
21427    2022-11-08T17:00:08Z
21428    2022-11-07T21:15:39Z
21429    2022-11-07T15:24:33Z
Name: published, Length: 21430, dtype: object

In [24]:
def publish_to_date(x):
    x = x.replace('Z',"")
    x = x.replace('T'," ")
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

In [26]:
df['published'] = df['published'].apply(publish_to_date)

In [46]:
def hours(x):
    y=x.hour
    return y

def days(x):
    y=x.weekday()
    return y

def months(x):
    y=x.month
    return y

In [47]:
hours = df['published'].apply(hours)
days = df['published'].apply(days)
months = df['published'].apply(months)

In [48]:
days.value_counts()

3    6560
2    4003
4    3467
1    3172
0    2233
6    1246
5     749
Name: published, dtype: int64

In [49]:
df_new_joined = pd.concat([hours,days,months], axis=1)
df_new_joined.columns = ['hours','days','months']

In [50]:
hours_in_day = 24
days_in_week = 7
months_in_year = 12

df_new_joined['sin_hours'] = np.sin(2*np.pi*(df_new_joined['hours']-1)/hours_in_day)
df_new_joined['cos_hours'] = np.cos(2*np.pi*(df_new_joined['hours']-1)/hours_in_day)
df_new_joined['sin_day_of_week'] = np.sin(2*np.pi*(df_new_joined['days']-1)/days_in_week)
df_new_joined['cos_day_of_week'] = np.cos(2*np.pi*(df_new_joined['days']-1)/days_in_week)
df_new_joined['sin_months'] = np.sin(2*np.pi*(df_new_joined['months']-1)/months_in_year)
df_new_joined['cos_months'] = np.cos(2*np.pi*(df_new_joined['months']-1)/months_in_year)

df_new_joined.drop(columns=['hours'], inplace=True)
df_new_joined.drop(columns=['days'], inplace=True)
df_new_joined.drop(columns=['months'], inplace=True)

In [56]:
df = df.join(df_new_joined)

In [63]:
df.category_id.isna().sum()

8

In [64]:
df.duration

0         238.0
1        2451.0
2        3027.0
3         170.0
4         830.0
          ...  
21425     197.0
21426      46.0
21427       NaN
21428       NaN
21429       NaN
Name: duration, Length: 21430, dtype: float64

In [65]:
from sklearn.impute import SimpleImputer
imputer_mean = SimpleImputer(strategy="mean") 
df['duration'] = imputer_mean.fit_transform(df[['duration']]) 
imputer_mode = SimpleImputer(strategy="most_frequent")
df['category_id'] = imputer_mode.fit_transform(df[['category_id']]) 
df

,Unnamed: 0.1,Unnamed: 0,id,channel_id,category_id,country,channel_viewCount,channel_subscriberCount,channel_videoCount,title,published,views,duration,sin_hours,cos_hours,sin_day_of_week,cos_day_of_week,sin_months,cos_months
0,0,0,RcFBkhJuoiU,UC9zY_E8mcAo_Oq772LEZq8Q,10.0,JP,2.575724e+09,7030000.0,481.0,yama - 色彩 / THE FIRST TAKE,2022-11-11 13:00:09,2253613,238.000000,1.224647e-16,-1.000000,0.433884,-0.900969,-0.866025,0.5
1,1,0,OGwygUDKOTQ,UC7Pq3Ko42YpkCB_Q4E981jw,10.0,US,1.226866e+10,20900000.0,3564.0,Learn Numbers + More Educational Videos &amp; ...,2022-11-11 12:00:16,297399,2451.000000,2.588190e-01,-0.965926,0.433884,-0.900969,-0.866025,0.5
2,2,0,MMf5Fz7Tn_I,UCsCy3274qvLzrbq9vGYLCpg,27.0,US,4.697463e+07,184000.0,96.0,Windstorm in the Desert +More | Super Rescue T...,2022-11-11 12:00:22,446318,3027.000000,2.588190e-01,-0.965926,0.433884,-0.900969,-0.866025,0.5
3,3,0,ur9yyQHbHvg,UCBSxVE6JoMg0WXvpm47OS3g,10.0,GB,8.593039e+08,4400000.0,63.0,Louis Tomlinson - Face The Music (Official Audio),2022-11-11 10:12:10,337291,170.000000,7.071068e-01,-0.707107,0.433884,-0.900969,-0.866025,0.5
4,4,0,6RfhCJJ5DgA,UCu59yAFE8fM0sVNTipR4edw,27.0,US,2.110474e+10,37400000.0,1008.0,Masha and the Bear 💥 NEW EPISODE 2022 💥 Someth...,2022-11-11 09:00:09,3249029,830.000000,8.660254e-01,-0.500000,0.433884,-0.900969,-0.866025,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21425,775,0,8Kn4yT4BdNg,UCHMrZNfXcc9VsOvINjd2E6g,10.0,NaN,7.700000e+02,3.0,36.0,Wildcoast Drive,2022-11-09 02:30:47,46,197.000000,2.588190e-01,0.965926,0.781831,0.623490,-0.866025,0.5
21426,776,0,yiD9KHPqoGM,UC5cq3LFBx3QFpVhW7nwJLwA,10.0,NaN,NaN,NaN,NaN,BeamNG Drive Crashes #032 #shorts #shortsvideo...,2022-11-09 00:30:13,378,46.000000,-2.588190e-01,0.965926,0.781831,0.623490,-0.866025,0.5
21427,777,0,iSMzxxNZjDo,UCBKSNxU0iWKmMfp_TtJHBAA,10.0,NaN,NaN,NaN,NaN,"Drive News | Tesla Model 3, Model X | Փարթ 1",2022-11-08 17:00:08,19703,822.326229,-8.660254e-01,-0.500000,0.000000,1.000000,-0.866025,0.5
21428,778,0,NhcDrbPYwLo,UCB8wO7_QO9eZOV63oATZ35A,10.0,NaN,NaN,NaN,NaN,NYC Cross-Borough Drive | New York City 4K,2022-11-07 21:15:39,6787,822.326229,-8.660254e-01,0.500000,-0.781831,0.623490,-0.866025,0.5
